In [0]:
from delta.tables import DeltaTable
from pyspark.sql import functions as F

In [0]:
dbutils.fs.mounts()

In [0]:
mount_path = "/mnt/my_adls_container"

product_df = spark.read.format("delta").load(f"{mount_path}/samples/product_delta_30_07_2025")
display(product_df)

In [0]:
product_df.count()

In [0]:
product_df = product_df.filter(F.col("SellEndDate").isNotNull())
product_df.write.format("delta").mode("overwrite").save(f"{mount_path}/samples/product_delta_30_07_2025")

In [0]:
spark.sql(f"describe history delta.`{mount_path}/samples/product_delta_30_07_2025`").display()

In [0]:
product_df = spark.read.format("delta").option("versionAsOf", 0).load(f"{mount_path}/samples/product_delta_30_07_2025")
display(product_df)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False)
])

data = [(i, f"Product_{i}") for i in range(1, 21)]

sample_df = spark.createDataFrame(data, schema)
display(sample_df)

In [0]:
sample_df.write.format("delta").mode("overwrite").save("dbfs:/FileStore/tables/delta_tables/product")

In [0]:
# Read the delta table

delta_product = DeltaTable.forPath(spark, "dbfs:/FileStore/tables/delta_tables/product")
delta_product.delete("name = 'Product_12'")

In [0]:
spark.sql("describe history delta.`dbfs:/FileStore/tables/delta_tables/product`").display()

In [0]:
delta_product.delete("name = 'Product_13'")

In [0]:
spark.sql("describe history delta.`dbfs:/FileStore/tables/delta_tables/product`").display()

In [0]:
%fs
ls dbfs:/FileStore/tables/delta_tables/product

In [0]:
# time travel

old_df = spark.read.format("delta").option("versionAsOf", 0).load("dbfs:/FileStore/tables/delta_tables/product")
old_df.display()

In [0]:
old_df = spark.read.format("delta").option("versionAsOf", 1).load("dbfs:/FileStore/tables/delta_tables/product")
old_df.display()